In [ ]:
%matplotlib widget
import sympy as sp
import numpy as np
import matplotlib.pylab as plt

In [ ]:
w = sp.Symbol(r'w', real=True)
s = sp.Symbol(r's', real = True)
d_a, E_c = sp.symbols(r'd_a, E_c', nonnegative=True)
f_t = sp.symbols(r'f_t', nonnegative=True)
c_2 = sp.Symbol('c_2', nonnegative=True)
c_1 = sp.Symbol('c_1', nonnegative=True)
f_c = sp.Symbol('f_c', nonnegative =True)
# a_3, a_4, r = f_c = sp.symbols('a_3, a_4, r', nonnegative =True)
# tau_0 = sp.Symbol(r'\tau_0', nonnegative =True)

In [ ]:
G_f = 0.028 * f_c ** 0.18 * d_a ** 0.32

L_cr = E_c * G_f / f_t ** 2

w_cr = f_t / E_c * L_cr

f_w = f_t * sp.exp(-f_t * w / G_f)

w_x = 5.14 * (G_f / f_t)


r = s / w

tau_0 = 0.3 * f_c

a_3 = 2.45 / tau_0

a_4 = 2.44 * (1 - (4 / tau_0))

In [ ]:
tau_s = sp.Piecewise(
        (0, w <= 0),
        (tau_0 * (1 - sp.sqrt((2 * w) / d_a)) * r * (a_3 + (a_4 * np.abs(r) ** 3)) / (1 + (a_4 * r ** 4)), w > 0)

    )
tau_s

In [ ]:
sigma_ag = sp.Piecewise(
        (0, w <= 0),
        (-0.62 * tau_s * (s) / (w**2 + r ** 2) ** 0.25, w > 0)
    )
sigma_ag

In [ ]:
sig_w = sp.Piecewise(
        (-f_c, w / L_cr <= - f_c / E_c),
        (E_c * w / L_cr, w <= w_cr),
#        (f_w, w <= w_x),
        (-sigma_ag, True)
    )

In [ ]:
sig_w

In [ ]:
params = {
    d_a : 22,
    E_c : 28000,
    f_t : 3,
    c_2 : 6,
    c_1 : 3,
    f_c : 30
}

In [ ]:
get_sig_w = sp.lambdify((w,s), sig_w.subs(params), 'numpy')
sig_w.subs(params)

In [ ]:
get_tau_s = sp.lambdify((w,s), tau_s.subs(params), 'numpy')
tau_s.subs(params)

In [ ]:
get_sig_ag = sp.lambdify((w,s), sigma_ag.subs(params), 'numpy')
sigma_ag.subs(params)

In [ ]:
_, (ax1) = plt.subplots(1, 1, figsize=(10,5))
w_ = 0.1
s_data = np.linspace(0 , 1.5, 100)
sig_ag = get_sig_ag(w_, s_data)
ax1.plot(s_data, sig_ag, color='blue', lw=2);

In [ ]:
_, (ax1) = plt.subplots(1, 1, figsize=(10,5))
w_ = 2
s_data = np.linspace(0 , 2, 100)
#w_data = np.linspace(0, 3, 100)
tau_s_val = get_tau_s(w_, s_data)
#tau_s_val = get_tau_s(w_data, 2)
ax1.plot(s_data, tau_s_val, color='blue', lw=2);
plt.show()

In [ ]:
_, (ax1) = plt.subplots(1, 1, figsize=(10,5))
w_ = np.linspace(-1,1, 100)
s_data = np.linspace(-1 , 1, 100)
sig = get_sig_w(w_, s_data)
ax1.plot(w_, sig, color='blue', lw=2);

In [ ]:
%matplotlib widget
fig = plt.figure()
ax3d = fig.add_subplot(1, 1, 1, projection='3d')
w_min = 1e-9  # -1
w_max = 3
w_data = np.linspace(w_min, w_max, 100)
s_max = 3
s_data = np.linspace(-1.1 * s_max, 1.1 * s_max, 100)  # -1.1
s_, w_ = np.meshgrid(s_data, w_data)
tau_s = get_tau_s(w_, s_)
ax3d.plot_surface(w_, s_, tau_s, cmap='viridis', edgecolor='none')
ax3d.set_xlabel(r'$w\;\;\mathrm{[mm]}$', fontsize=12)
ax3d.set_ylabel(r'$s\;\;\mathrm{[mm]}$', fontsize=12)
ax3d.set_zlabel(r'$\tau\;\;\mathrm{[MPa]}$', fontsize=12)
ax3d.set_title('aggregate interlock law', fontsize=12)


In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
w_data = np.linspace(0.1, 1, 10)
s_data = np.linspace(0, 2.5, 100)  # -1.1
for w_ in w_data:
    tau_s = get_tau_s(w_, s_data)
    ax.plot(s_data, tau_s, color='blue', label='w = {}'.format(w_))


In [ ]:
%matplotlib widget
fig = plt.figure()
ax3d = fig.add_subplot(1, 1, 1, projection='3d')
w_min = 1e-9  # -1
w_max = 3
w_data = np.linspace(w_min, w_max, 100)
s_max = 6
s_data = np.linspace(-1.1 * s_max, 1.1 * s_max, 100)  # -1.1
s_, w_ = np.meshgrid(s_data, w_data)
tau_s = get_sig_w(w_, s_)
ax3d.plot_surface(w_, s_, tau_s, cmap='viridis', edgecolor='none')
ax3d.set_xlabel(r'$w\;\;\mathrm{[mm]}$', fontsize=12)
ax3d.set_ylabel(r'$s\;\;\mathrm{[mm]}$', fontsize=12)
ax3d.set_zlabel(r'$\tau\;\;\mathrm{[MPa]}$', fontsize=12)
ax3d.set_title('aggregate interlock law', fontsize=12)


In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
w_data = np.linspace(0.1, 1, 300)
s_data = np.linspace(0, 2.5, 300)  # -1.1
for w_ in w_data:
    tau_s = get_sig_w(w_data, 3)
    ax.plot(w_data, tau_s, color='blue', label='w = {}'.format(w_))
